Unitary designs are used when there is a need for random sampling or randomizations. More precisely, the Haar measure provides a way to sample uniformly from the group of unitary matrices, but generating truly random unitaries with respect to the Haar measure can be computationally expensive. Unitary designs offer an efficient way to approximate this randomness. A unitary design is a collection of unitary matrices that, when averaged together, behaves like an integral over the Haar measure.
The 2-desings specifically are used for operation benchmarking on qubits. If an operation is perfectly act on a qubit state as $\Lambda (|0\rangle \langle0|)$ it could be expressed as $V|0\rangle \langle0|V^{\dagger}$. And the fidelity measures their overlapping, i.e. $F = \langle 0|V^{\dagger} \Lambda(|0\rangle \langle0|) V |0\rangle $ 

$\bar F(\Lambda, V) = \int \, d\mu(U) \langle 0 | U^\dagger V^\dagger \Lambda (U |0\rangle \langle 0 | U^\dagger) V U |0 \rangle$


The above expressions defines the average fidelity over the unitaries by using Haar measure. This is exactly the case where the unitary t-designs come handy. The equation can be expressed in an exact form as 

$ \int \, d\mu(U) \langle 0 | U^\dagger V^\dagger \Lambda (U |0\rangle \langle 0 | U^\dagger) V U |0 \rangle = \frac{1}{K} \sum\limits_{j=1}^{K} \langle 0 | U_j^\dagger V^\dagger \Lambda (U_j |0\rangle \langle 0 | U_j^\dagger) V U_j |0 \rangle$


Here $K$ is the cardinality of the set of unitaires that form a 2-design.

It can be shown that the Clifford goup is a unitary-3 design so it is a 2 and 1 design, too. For the 1-qubit Clifford group we have the Hadamard ($H$) gate and the phase ($S$) gate as the generator of the Clifford group. Altogether the group consists of 24 elements. So over this 24 elements we need to perform the summation written above.

In [6]:
single_qubit_cliffords = [
 '',
 'H', 'S',
 'HS', 'SH', 'SS',
 'HSH', 'HSS', 'SHS', 'SSH', 'SSS',
 'HSHS', 'HSSH', 'HSSS', 'SHSS', 'SSHS',
 'HSHSS', 'HSSHS', 'SHSSH', 'SHSSS', 'SSHSS',
 'HSHSSH', 'HSHSSS', 'HSSHSS'
]

Here we will check if using the Clifford group for the evaluation of the average fidelity gives the same result as the definition, i.e. integrating over the Haar measure by using a number of randomly sampled unitaries.

In [7]:
import pennylane as qml
import numpy as np

# Scipy for sampling Haar-random unitaries
from scipy.stats import unitary_group

# set the random seed
np.random.seed(42)

# Use the mixed state simulator
dev = qml.device("default.mixed", wires=1)

In [8]:
# We set up the experiment by defining an operation which we hit by some noise.
# This will make the fidelity diverge from unity for certain.

def ideal_experiment():
    qml.SX(wires=0)
    return qml.state()


def noisy_operations(damp_factor, depo_factor, flip_prob):
    qml.AmplitudeDamping(damp_factor, wires=0)
    qml.DepolarizingChannel(depo_factor, wires=0)
    qml.BitFlip(flip_prob, wires=0)


@qml.qfunc_transform
def apply_noise(tape, damp_factor, depo_factor, flip_prob):
    # Apply the original operations
    for op in tape.operations:
        qml.apply(op)

    # Apply the noisy sequence
    noisy_operations(damp_factor, depo_factor, flip_prob)

    # Apply the original measurements
    for m in tape.measurements:
        qml.apply(m)

In [9]:
# the experiment with noise is created
damp_factor = 0.02
depo_factor = 0.02
flip_prob = 0.01

noisy_experiment = apply_noise(damp_factor, depo_factor, flip_prob)(ideal_experiment)

In [10]:
# Defining the conjugation with a unitary
@qml.qfunc_transform
def conjugate_with_unitary(tape, matrix):
    qml.QubitUnitary(matrix, wires=0)

    for op in tape.operations:
        qml.apply(op)

    qml.QubitUnitary(matrix.conj().T, wires=0)

    for m in tape.measurements:
        qml.apply(m)

In [12]:
# defining fidelity
from scipy.linalg import sqrtm

def fidelity(rho, sigma):
    # Inputs rho and sigma are density matrices
    sqrt_sigma = sqrtm(sigma)
    fid = np.trace(sqrtm(sqrt_sigma @ rho @ sqrt_sigma))
    return fid.real

In [64]:
from scipy.linalg import sqrtm

def fidelity(rho, sigma):
    # Inputs rho and sigma are density matrices
    sqrt_sigma = sqrtm(sigma)
    fid = np.trace(sqrtm(sqrt_sigma @ rho @ sqrt_sigma))
    return fid.real

In [65]:
def computing_average_fidelity(ideal_exp,noisy_exp,unitary=True):
    n_samples = 1000
    fidelities = []
    
        # Apply transform to construct the ideal and noisy quantum functions
    if unitary:
        U = unitary_group.rvs(2)
        for _ in range(n_samples):
            conjugated_ideal_experiment = conjugate_with_unitary(U)(ideal_exp)
            conjugated_noisy_experiment = conjugate_with_unitary(U)(noisy_exp)
            # Use the functions to create QNodes
            ideal_qnode = qml.QNode(conjugated_ideal_experiment, dev)
            noisy_qnode = qml.QNode(conjugated_noisy_experiment, dev)

            # Execute the QNodes
            ideal_state = ideal_qnode()
            noisy_state = noisy_qnode()

            # Compute the fidelity
            fidelities.append(fidelity(ideal_state, noisy_state))

    else:
        for C in single_qubit_cliffords:
            conjugated_ideal_experiment = conjugate_with_clifford(C)(ideal_exp)
            conjugated_noisy_experiment = conjugate_with_clifford(C)(noisy_exp)
            # Use the functions to create QNodes
            ideal_qnode = qml.QNode(conjugated_ideal_experiment, dev)
            noisy_qnode = qml.QNode(conjugated_noisy_experiment, dev)

            # Execute the QNodes
            ideal_state = ideal_qnode()
            noisy_state = noisy_qnode()

            # Compute the fidelity
            fidelities.append(fidelity(ideal_state, noisy_state))
            
    return np.mean(fidelities)

In [66]:
computing_average_fidelity(ideal_experiment, noisy_experiment)

0.9839931674870752

In [67]:
# we define the action of the Clifford group elements using their string representation
def apply_single_clifford(clifford_string, inverse=False):
    for gate in clifford_string:
        #if a H string we apply Hadamard
        if gate == 'H':
            qml.Hadamard(wires=0)
        #if S string we apply phase flip
        else:
            sign = -1 if inverse else 1
            qml.PhaseShift(sign * np.pi/2, wires=0)

In [68]:
#just as before we define the conjugation but this time with Clifford elements
@qml.qfunc_transform
def conjugate_with_clifford(tape, clifford_string):
    #applying the Clifford
    apply_single_clifford(clifford_string, inverse=False)

    for op in tape.operations:
        qml.apply(op)
    #applying Clifford conjugate
    apply_single_clifford(clifford_string, inverse=True)

    for m in tape.measurements:
        qml.apply(m)

In [69]:
computing_average_fidelity(ideal_experiment, noisy_experiment,unitary=False)

0.9867892193517145

Lets compare the two methods one with Haar-sampled unitaries the other with the 2-design Clifford group.

In [70]:
print(
    "Using unitaries:", computing_average_fidelity(ideal_experiment, noisy_experiment,unitary=True),
    "\n"
    "Using 2-designs:", computing_average_fidelity(ideal_experiment, noisy_experiment,unitary=False)
    )

Using unitaries: 0.9895457714257622 
Using 2-designs: 0.9867892193517145


The agreement could be further enhanced by using more Haar-sampled unitaries.